In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
from zero_helpers.imports import * 

In [12]:
from zero_liftsim.simmanager import SimulationManager
from zero_liftsim.experience import AgentRideLoopExperience
import inspect
from zero_liftsim.helpers import load_asset_template

# Run Simulation

In [13]:
# setup and run simulation
manager = SimulationManager(
    n_agents=10,
    lift_capacity=2,
)
result = manager.run()
print(result)

{'total_rides': 323, 'average_wait': 1.671826625386997, 'agents': [Agent(1) ripe-jacket c866615b-7fdc-4b9b-9e3a-81e254f3f655, Agent(2) half-kill 64340ced-0b09-4353-a412-aa9e5d2709c5, Agent(3) wandering-item 8fa05aea-31ce-4607-9a30-ff8c7729559a, Agent(4) large-virus 9f6c8249-7fa5-40cb-8c7a-1fbce5346ad3, Agent(5) uneven-external 17542903-00b8-448d-b6e2-e36e12b86287, Agent(6) black-problem 6580b185-c180-4299-9eed-ac564480b21a, Agent(7) testy-benefit 0373ba94-d543-4cdb-a5fa-c5ede8b5d7cd, Agent(8) medical-give f3c5ce78-f7fa-4bf6-81a5-c38300a5389a, Agent(9) heavenly-power 697a0f8e-880a-4fdb-8685-2c24bd17ad50, Agent(10) safe-woman fa5f5048-21c4-4d2c-b2ed-17cdd786ddd7]}


In [14]:
lift = manager.lift
lift.total_chairs()

50

## Retrieve Data from Simulation

In [15]:
# get rideloop explogs and agent event log
exp_log_data = manager.retrieve_data()
e = exp_log_data['exp_rideloop']
l = exp_log_data['agent_log']

# Identify Agent State

In [16]:
# get ride loop logs and agent event logs
log = manager.retrieve_data()
rlog = log['exp_rideloop'] # dataframe of all agent rideloop experiences 
alog = log['agent_log'] # combined agent logs (i.e. activity logs for all agents)

# sample a random timestamp within the agent log timeframe
INDEX_TIME = pd.Series(pd.date_range(alog['time'].min(), alog['time'].max(), freq='1s')).sample().iloc[0]

# sample a random agent from ride loop log
row = rlog.sample().iloc[0]
aid = row['agent_uuid']
agent = manager.lookup_agent(aid)

# find the nearest time index in the agent log
l = alog.zero_liftsim.subset_agent_uuid(agent.agent_uuid)
e = rlog.zero_liftsim.subset_agent_uuid(agent.agent_uuid)
idx = l.zero_liftsim.get_nearest_time_index(INDEX_TIME)

for the ride loop agent experience, the total time is broken down into riding the lift, traversing down the mountain and waiting in the queue, i.e. `"time_spent_total": ride + traverse + queue` below: 

In [17]:
s = inspect.getsource(AgentRideLoopExperience)
lines = s.split('\n')
idx = [i for i,x in enumerate(lines) if x.strip() == '"""'][-1]
print('\n'.join(lines[idx+1:]))

    def add_entry(self, agent, return_event_uuid, dt: datetime, ride: float, traverse: float, 
                  queue: float) -> None:
        self.log[dt] = {
            "exp_id": str(uuid()), 
            "return_event_uuid": return_event_uuid,
            "agent_uuid": agent.agent_uuid,
            "agent_uuid_codename": agent.agent_uuid_codename,
            "time_spent_ride_lift": ride,
            "time_spent_traverse_down_mountain": traverse,
            "time_spent_in_queue": queue,
            "time_spent_total": ride + traverse + queue,
        }



..which means that we can define three states - state_riding_lift, state_in_queue and state_traversing_down. 

In [23]:
from zero_liftsim import sandbox as sb
states = sb.infer_agent_states(manager.agents, INDEX_TIME)
states

{'c866615b-7fdc-4b9b-9e3a-81e254f3f655': 'riding_lift',
 '64340ced-0b09-4353-a412-aa9e5d2709c5': 'traversing_down',
 '8fa05aea-31ce-4607-9a30-ff8c7729559a': 'in_queue',
 '9f6c8249-7fa5-40cb-8c7a-1fbce5346ad3': 'in_queue',
 '17542903-00b8-448d-b6e2-e36e12b86287': 'in_queue',
 '6580b185-c180-4299-9eed-ac564480b21a': 'riding_lift',
 '0373ba94-d543-4cdb-a5fa-c5ede8b5d7cd': 'traversing_down',
 'f3c5ce78-f7fa-4bf6-81a5-c38300a5389a': 'traversing_down',
 '697a0f8e-880a-4fdb-8685-2c24bd17ad50': 'in_queue',
 'fa5f5048-21c4-4d2c-b2ed-17cdd786ddd7': 'traversing_down'}